In [ ]:
import shutil

try:
  shutil.rmtree('custom_q_and_a')
except:
  print("Folder doesnt exist")

In [ ]:
import json

input_file = "addsub_training.json"  # Replace with the path to your JSONL file
output_file = "output.txt"  # Output file path

with open(input_file, "r") as f:
    data = json.load(f)

x = len(data)*0.7
count = 0
with open(output_file, "w") as f:
    for question in data:
        if count>=x:
          break
        # json_data = json.loads(line.strip())
        prompt = question["prompt"]
        completion = question["reasoning"]
        answer = question["answer"]
        f.write(f"[Q] {prompt}\n")
        # f.write("[Q] " + prompt + "\n")
        f.write(f"[A] {completion}. Answer \\boxed{{{answer}}}.--> {answer} END\n")
        # f.write("[A] " + completion + " --> " + answer  " END\n")
        f.write("\n")
        count+=1

In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory():
    combined_text = ""
    file_path = 'output.txt'
    combined_text += read_txt(file_path)
    return combined_text

In [ ]:
# train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/q_and_a'
text_data = read_documents_from_directory()
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters
text_data = re.sub(r"END", "END\n",text_data)
text_data

In [ ]:
with open(output_file, 'w') as output_file:
    # output_file.write("")
    output_file.write(text_data)
    output_file.close()

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
train_file_path = "output.txt"
model_name = 'gpt2'
output_dir = 'custom_q_and_a'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50
save_steps = 50000

In [ ]:
!pip install accelerate -U

In [ ]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.942700
1000,0.358200
1500,0.182800
2000,0.116000
2500,0.088900


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    model.eval()
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return tokenizer.decode(final_outputs[0], skip_special_tokens=True)

In [ ]:
model2_path = "custom_q_and_a"
sequence2 = "[Q] There are 37 short bushes and 30 tall trees currently in the park . Park workers will plant 20 short bushes today . How many short bushes will the park have when the workers are finished ?"
max_len = 200
answer = generate_text(model2_path, sequence2, max_len)
print(answer)

[Q] There are 37 short bushes and 30 tall trees currently in the park. Park workers will plant 20 short bushes today. How many short bushes will the park have when the workers are finished?
[A] Sure! Let's break down the problem step by step:
1. Currently, there are 37 short bushes and 30 tall trees in the park, for a total of 85 trees.
2. Today, park workers will plant 20 short bushes.
3. To find out how many short bushes the park will have after the workers are finished, we need to add the number of short bushes being planted (35) to the current number of short bushes (87):
87 + 20 = 98
So, after the workers are finished, there will be 98 short bushes in the park.
Is that correct?. Answer: \boxed{98}.--> 98 END

[Q] There are currently 37 short bushes and 30 tall trees in the park.


In [ ]:
f = open('addsub_training.json')
test_data = json.load(f)

In [ ]:
correct_predictions = 0
model2_path = "custom_q_and_a"
max_len = 200
for i in range(int(len(test_data)*0.7), len(test_data)):

  new_prompt = test_data[i]['prompt']
  answer = generate_text(model2_path, "[Q] "+new_prompt, max_len)
  try:
    pred = int(answer.split("-->")[1].split("END")[0].strip())
  except:
    pred = -9999
    pass
  ans = test_data[i]['answer']
  if '.' in test_data[i]['answer']:
    pred=float(pred)
    ans = float(ans)
  else:
    ans = int(ans)
  if pred == ans:
    correct_predictions+= 1


In [ ]:
total_data_points = len(test_data) - int(len(test_data)*0.7)
print("Accuracy = ",(correct_predictions/total_data_points)*100, "%")

Accuracy =  12.605042016806722 %
